In [1]:
import plotly.plotly as py
import plotly
import plotly.graph_objs as go
import pandas as pd
from matplotlib.colors import ListedColormap
import json
import seaborn as sns
import numpy as np

plotly.tools.set_credentials_file(username='luke_martin', api_key='dwE475ETUXR0Yra829jp')

### Load data

In [2]:
data = json.load(open("rnb_data.json"))

features = ['danceability',          \
                'energy',            \
                'speechiness',       \
                'acousticness',      \
                'tempo',             \
                'instrumentalness',  \
                'liveness',          \
                'valence']
all_genres = ['contemporary-rnb', 'british-rnb', 'alternative-rnb', 'new-jack-swing-rnb', 'new-orleans-rnb', 'jump-blues', 
             'blue-eyed-soul', 'early-rnb', 'motown-rnb']

### Filter json data

In [10]:
data_years = set([d['year'] for d in data])
print(data_years)

new_data = []

for y in data_years :
    
    subdata = [d for d in data if d['year']==y]
    albums = [d['album'] for d in subdata]
    nb_album = len(set(albums))
    to_retrieve = 800
    
    #too many data
    if (len(albums) > 800) :
        
        filtered_data = []
        
        for a in albums :
            if nb_album >= 1 :
                nb_per_album = int(800/nb_album)
                album_data = [d for d in data if d['album']==a]
                to_add = album_data[:nb_per_album]
                filtered_data.extend(to_add)
                nb_album -=1
                to_retrieve -= len(to_add)
            
        new_data.extend(filtered_data)
            
        
        
json.dump(new_data, open("filtered_data.json", 'w'))

{'1962', '2002', '1991', '2016', '1968', '1971', '1999', '2015', '2006', '1987', '1970', '1965', '1960', '2003', '1998', '1959', '1900', '1975', '2012', '2004', '1955', '1995', '1985', '1976', '1988', '1958', '2001', '1981', '1957', '2009', '1983', '1961', '1982', '1964', '1986', '2018', '1994', '1979', '1980', '1989', '2014', '1966', '1973', '2010', '1969', '2013', '1967', '1992', '2011', '1990', '1972', '2008', '1996', '2007', '2005', '1997', '1984', '1974', '1993', '2000', '2017', '1963', '1956', '1977', '1978'}


### Comparative bar plots

In [3]:
def compare_genre_bars(genres) :

    genred_data = [[d for d in data if d['genre'] == genre] for genre in genres]
    print(list(zip(genres, [len(c) for c in genred_data])))
    traces = []
    for e, g in enumerate(genres) :
        
        scores = []
        for f in features :
            if f == 'tempo' :
                scores.append(np.mean(np.array([d[f] for d in genred_data[e]])))
            else :
                scores.append(np.mean(np.array([d[f]*100 for d in genred_data[e]])))
            
        traces.append(go.Bar(x=features, y=scores, name=g))
        
    title = 'Features of ' + genres[0].replace('-', ' ') + ' and ' + genres[1].replace('-', ' ')
    layout = go.Layout(barmode='group', title=title)
    fig = go.Figure(data=traces, layout=layout)
    py.iplot(fig, sharing='public', filename=title)

In [4]:
compare_genre_bars(['early-rnb', 'new-orleans-rnb'])
compare_genre_bars(['jump-blues', 'new-orleans-rnb'])
compare_genre_bars(['early-rnb', 'new-jack-swing-rnb'])
compare_genre_bars(['contemporary-rnb', 'british-rnb'])
compare_genre_bars(['contemporary-rnb', 'alternative-rnb'])


[('early-rnb', 3327), ('new-orleans-rnb', 3014)]


/home/luke/anaconda3/lib/python3.6/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



[('jump-blues', 2094), ('new-orleans-rnb', 3014)]
[('early-rnb', 3327), ('new-jack-swing-rnb', 1764)]
[('contemporary-rnb', 9723), ('british-rnb', 1640)]
[('contemporary-rnb', 9723), ('alternative-rnb', 4353)]


### Features study

In [5]:
def tuple_to_rgb(color_tuple) :  
    
    def change_col_fmt(x) :
        return str(int(255*x))
    
    return "rgb("+ ','.join([change_col_fmt(color_tuple[c]) for c in [0, 1, 2]]) + ')'


def feature_study(feature) :
    
    ys = []
    for g in all_genres :
        ys.append(np.mean(np.array([d[feature] for d in data if d['genre'] == g])))
      
    ys = np.array(ys)
    sort_indices = np.argsort(ys)
    ys = ys[sort_indices]
    sorted_genres = np.array(all_genres.copy())[sort_indices]

    
    cmap = sns.cubehelix_palette(len(all_genres))
    trace = go.Bar(x=sorted_genres, 
                   y=ys, 
                   marker=dict(color=[tuple_to_rgb(c) for c in sns.color_palette("Set2")]))
    title = 'Mean of ' + feature + ' value in songs for main RnB genres'
    layout = go.Layout(title=title, yaxis=dict(title=feature))
    fig = go.Figure(data=[trace], layout=layout)
    py.iplot(fig, sharing='public', filename=title)

In [6]:
for f in features :
    feature_study(f)

### Album comparisons

In [7]:
classic_cont  = json.load(open("classic_album_cont.json"))
classic_no    = json.load(open("classic_album_no.json"))
classic_early = json.load(open("classic_album_early.json"))
classic_brit  = json.load(open("classic_album_british.json"))

In [8]:
def compare_album_bars(albums_dicts) :

    traces = []
    for e, g in enumerate(albums_dicts) :        
        scores = []
        for f in features :
            if f == 'tempo' :
                scores.append(g[f])
            else :
                scores.append(g[f]*100)
            
        traces.append(go.Bar(x=features, y=scores, name=g['album']))
        
    title = 'Features of ' + albums_dicts[0]['album'].replace('-', ' ') + ' and ' + albums_dicts[1]['album'].replace('-', ' ')
    layout = go.Layout(barmode='group', title=title)
    fig = go.Figure(data=traces, layout=layout)
    py.iplot(fig, sharing='public', filename=title)

In [9]:
compare_album_bars([classic_cont, classic_early, classic_no, classic_brit])